### Installations and imports

In [2]:
CMD = "wget https://raw.githubusercontent.com/Diyago/"\
  "GAN-for-tabular-data/master/requirements.txt"

!{CMD}
!pip install -r requirements.txt
!pip install tabgan

--2023-03-14 16:49:54--  https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     183  --.-KB/s    in 0s      

2023-03-14 16:49:54 (11.7 MB/s) - ‘requirements.txt’ saved [183/183]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
import pandas  as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import xgboost
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import time
import statistics as st
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
from tabgan.sampler import GANGenerator
import warnings
warnings.filterwarnings('ignore')

### Data loading and pre-processing

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/sydney-machine-learning/review-classimbalanced-ensemblelearning/main/Datasets/Ecoli%20Datasets/Ecoli4.csv', sep=",", header='infer' )

In [5]:
df.head()

,Unnamed: 0,one,two,three,four,five,six,seven,eight,label
0,0,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,Positive
1,1,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,Positive
2,2,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,Positive
3,3,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,Positive
4,4,ADI_ECOLI,0.23,0.32,0.48,0.5,0.55,0.25,0.35,Positive


In [6]:
y = df['label']
y.value_counts()

Positive    316
negative     20
Name: label, dtype: int64

In [7]:
df = df.drop(['Unnamed: 0'], axis = 1)
df.shape

(336, 9)

In [8]:
lencoders = {}
for col in df.select_dtypes(include=['object']).columns:
    lencoders[col] = LabelEncoder()
    df[col] = lencoders[col].fit_transform(df[col])

In [9]:
y = df['label']
X = df.drop(['label'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = 42)

 ### CT-GAN

In [10]:
# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(X, y, test_size=0.4, random_state = 42)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

# Build the neural network
model = Sequential()
# Hidden 1
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu')) 
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

Epoch 1/1000
7/7 - 6s - loss: 16.4582 - val_loss: 11.6526 - 6s/epoch - 878ms/step
Epoch 2/1000
7/7 - 0s - loss: 4.4195 - val_loss: 3.0527 - 50ms/epoch - 7ms/step
Epoch 3/1000
7/7 - 0s - loss: 3.0213 - val_loss: 0.0732 - 117ms/epoch - 17ms/step
Epoch 4/1000
7/7 - 0s - loss: 0.9246 - val_loss: 0.7690 - 98ms/epoch - 14ms/step
Epoch 5/1000
7/7 - 0s - loss: 0.3238 - val_loss: 0.6568 - 169ms/epoch - 24ms/step
Epoch 6/1000
7/7 - 0s - loss: 0.3459 - val_loss: 0.2014 - 116ms/epoch - 17ms/step
Epoch 7/1000
7/7 - 0s - loss: 0.2667 - val_loss: 0.0597 - 64ms/epoch - 9ms/step
Epoch 8/1000
7/7 - 0s - loss: 0.1367 - val_loss: 0.0845 - 47ms/epoch - 7ms/step
Epoch 9/1000
7/7 - 0s - loss: 0.0766 - val_loss: 0.0924 - 46ms/epoch - 7ms/step
Epoch 10/1000
7/7 - 0s - loss: 0.0763 - val_loss: 0.0756 - 64ms/epoch - 9ms/step
Epoch 11/1000
7/7 - 0s - loss: 0.0696 - val_loss: 0.0642 - 102ms/epoch - 15ms/step
Epoch 12/1000
Restoring model weights from the end of the best epoch: 7.
7/7 - 0s - loss: 0.0611 - val_loss

In [11]:
gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           gan_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,\
          df_x_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)

Fitting CTGAN transformers for each column:   0%|          | 0/9 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

### CT_GAN + Ensembles Learning Techniques

In [15]:
dtc =  DecisionTreeClassifier()
rfc = RandomForestClassifier()
knn =  KNeighborsClassifier()
xgb = xgboost.XGBClassifier()
clf1 = [('dtc',dtc),('rfc',rfc),('knn',knn),('xgb',xgb)]
clf2 = [('rfc',rfc),('xgb',xgb)]
lr = LogisticRegression()

classi = [lightgbm.LGBMClassifier(), AdaBoostClassifier(), xgboost.XGBClassifier(),
          GradientBoostingClassifier(), DecisionTreeClassifier(), RandomForestClassifier(), 
          VotingClassifier(estimators=[('rfc',rfc),('xgb',xgb)], voting='soft'),
          VotingClassifier(estimators=[('rfc',rfc),('xgb',xgb)], voting='hard'),
          StackingClassifier( estimators = clf1,final_estimator = lr),
          StackingClassifier( estimators = clf2,final_estimator = lr)]

classi_lst = ['LGBM Classifier', 'AdaBoost Classifier', 'XGB Classifier', 'Gradient Boosting Classifier',
              'Decision Tree Classifier', 'Random Forest Classifier', 'Voting Classifier Soft',
              'Voting Classifier Hard', 'Stacking Classifier 1', 'Stacking Classifier 2']


X_train_res = gen_x
y_train_res = gen_y
        
acc_list = []
f1_list = []
roc_list = []

for j in range(0, 10):
            print("#######", classi_lst[j], "#######")
            classifier = classi[j]
            for k in range(30):
                classifier.fit(X_train_res, y_train_res)
                # make predictions for test data
                y_pred = classifier.predict(X_test)
        
                acc_list.append(accuracy_score(y_test, y_pred))
                f1_list.append(f1_score(y_test, y_pred))
                roc_list.append(roc_auc_score(y_test, y_pred))
      
            print("Accuracy - ", st.mean(acc_list), "(", max(acc_list), ",", st.stdev(acc_list),")")
            print("F1-score - ", st.mean(f1_list), "(", max(f1_list), ",", st.stdev(f1_list),")")
            print("ROC - ", st.mean(roc_list), "(", max(roc_list), ",", st.stdev(roc_list),")")

####### LGBM Classifier #######
Accuracy -  1.0 ( 1.0 , 0.0 )
F1-score -  1.0 ( 1.0 , 0.0 )
ROC -  1.0 ( 1.0 , 0.0 )
####### AdaBoost Classifier #######
Accuracy -  0.9851851851851852 ( 1.0 , 0.014939836565618118 )
F1-score -  0.8333333333333333 ( 1.0 , 0.1680731613632036 )
ROC -  0.875 ( 1.0 , 0.12605487102240268 )
####### XGB Classifier #######
Accuracy -  0.9827160493827161 ( 1.0 , 0.012660705364831592 )
F1-score -  0.811965811965812 ( 1.0 , 0.14017843529183321 )
ROC -  0.8541666666666666 ( 1.0 , 0.1068247015157664 )
####### Gradient Boosting Classifier #######
Accuracy -  0.9777777777777777 ( 1.0 , 0.013916095357447787 )
F1-score -  0.7756410256410257 ( 1.0 , 0.13670387175278523 )
ROC -  0.8417814960629921 ( 1.0 , 0.09486152146560507 )
####### Decision Tree Classifier #######
Accuracy -  0.9730864197530864 ( 1.0 , 0.015740801664818515 )
F1-score -  0.7444180492709904 ( 1.0 , 0.13774309258147308 )
ROC -  0.8334317585301837 ( 1.0 , 0.08642278379016265 )
####### Random Forest Classifi